In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
def data_load(path, cancers):
  import tarfile
  import os
  import pandas as pd
  from selenium import webdriver
  from bs4 import BeautifulSoup
  from selenium.webdriver.common.keys import Keys
  from urllib.request import urlopen
  from urllib.parse import quote_plus

  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome('chromedriver', options=chrome_options)

  genome_path = f'{path}../해제파일/' # 압축 해제 후 파일
  cancer_name = cancers  #입력 받는 암 데이터 명

  for i in cancers: # 데이터 다운로드
    url = 'http://gdac.broadinstitute.org/runs/stddata__2016_01_28/data/{0}/20160128/gdac.broadinstitute.org_{0}.Merge_rnaseqv2__illuminahiseq_rnaseqv2__unc_edu__Level_3__RSEM_genes_normalized__data.Level_3.2016012800.0.0.tar.gz'.format(i)
    driver.get(url)

  import shutil
  for c in cancers: # 데이터 경로 이동
    shutil.move('/content/gdac.broadinstitute.org_{0}.Merge_rnaseqv2__illuminahiseq_rnaseqv2__unc_edu__Level_3__RSEM_genes_normalized__data.Level_3.2016012800.0.0.tar.gz'.format(i),path)
  
  tar_list = os.listdir(path) #압축 해제 경로
  cancer_df = pd.DataFrame()
  for idx,f in enumerate(tar_list):
    genome = cancer_name[idx]
    ap = tarfile.open(path + f) #압축해제
    ap.extractall(genome_path) # 압축파일 저장
    ap.close
  
    f_old = os.path.join(genome_path, f.split('.tar')[0]) 
    f_new = os.path.join(genome_path, f'{genome}')
    os.rename(f_old, f_new) # 폴더 명 암 이름으로 요약)

    txt_list = os.listdir(f'{genome_path}{genome}/')
    file = ''.join([i for i in txt_list if str(genome) in i])

    data = pd.read_csv(f'{genome_path+genome}/{file}', sep='\t', low_memory=False, index_col='Hybridization REF', skiprows=[1])  
    data = data.T

    cancer_df = pd.concat([cancer_df, data])

  return cancer_df